In [ ]:
import csv
from cv2 import imread,cvtColor
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
file = open('driver_imgs_list.csv')
lst = csv.reader(file)
file_list = []
for i in lst:
    file_list.append(i)
file_list.pop(0)
w = 96
h = 128

In [ ]:
def read_file(file):
    img = cv2.imread('train/'+file[1]+'/'+file[2],0)
    img = cv2.resize(img,(w,h), interpolation=cv2.INTER_LINEAR)
    img = img.astype(np.float32)/255.0
    label = int(file[1][1])
    return img.reshape([1,-1]),label
def show(array):
    plt.figure()
    plt.imshow(array.reshape([h,w]), cmap="gray")
    plt.show()
def get_batch(batch_size):
    j = np.random.randint(0,20788,batch_size)
    batch = np.zeros((batch_size,w*h))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k:k+1,:],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label
def get_test_batch(batch_size):
    j = np.random.randint(20788,22424,batch_size)
    batch = np.zeros((batch_size,w*h))
    k = 0
    label = np.zeros((batch_size,10))
    for i in j:
        batch[k:k+1,:],lab = read_file(file_list[i])
        label[k,lab] = 1
        k+=1
    return batch,label

In [ ]:
num_steps = 500000000
lr = 0.001
display_step = 100
batch_size = 64

In [ ]:
h_3_size = 1000
input_size = w * h

In [ ]:
x = tf.placeholder (dtype=tf.float32,shape=[None, input_size])

In [ ]:
h_2 = tf.layers.dense(inputs=x,units=1000,activation=tf.nn.relu)

h_1 = tf.layers.dense(inputs=h_2,units=500,activation=tf.nn.relu)

d_1 = tf.layers.dense(inputs=h_1, units=1000, activation=tf.nn.relu)
d_2 = tf.layers.dense(inputs=d_1, units=96*128)
d_3 = tf.nn.sigmoid(d_2)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=d_2))
lrr = tf.placeholder(shape=[],dtype=tf.float32)
optimizer = tf.train.AdamOptimizer(learning_rate=lrr).minimize(loss)


In [ ]:
Y = tf.placeholder (dtype=tf.float32,shape=[None, 10])
y_1 = tf.layers.dense(inputs=h_1,units=500,name='alp',activation=tf.nn.relu)
y_11 = tf.layers.dense(inputs=y_1,units=100,name='alp1',activation=tf.nn.relu)
y_2 = tf.layers.dense(inputs=y_11,units=10,name='alp2')
clf_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=y_2))
optimizer2 = tf.train.AdamOptimizer(lrr).minimize(clf_loss)
correctPred = tf.equal(tf.argmax(y_2,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
loss_array = []
clf_array = []

In [ ]:
# Start Training# Training
for i in range(1, num_steps+1):
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    batch_x,batch_y = get_batch(64)

    #Run optimization op (backprop) and cost op (to get loss value)
    _, l,img = sess.run([optimizer, loss,d_3], feed_dict={x: batch_x,lrr:lr})
    loss_array.append(l)
    print('Recons:',l)
    batch_x,batch_y = get_batch(64)
    _, l= sess.run([optimizer2, clf_loss], feed_dict={x: batch_x,Y:batch_y,lrr:lr})
    clf_array.append(l)
    print('Class:',l)
    if i % display_step == 0 or i == 1:
        print('Step %i: Minibatch Loss: %f' % (i, l))
        lr *= 0.9
        show(batch_x[0:1,:])
        show(img[0:1,:])
    if lr < 0.00001:
        break

In [ ]:
total_acc = 0
for i in range(100):
    batch_x,label = get_test_batch(batch_size)
    l,acc = sess.run([clf_loss,accuracy], feed_dict={x: batch_x,Y:label})
    total_acc += acc
print(total_acc/100)

In [ ]:
plt.semilogy(loss_array,label='Recons')
plt.semilogy(clf_array,label='Class')
plt.legend()
plt.show()

In [ ]:
lr = 0.001